<b>Назначение:</b> <br>
Разбиение датасета на тренировочную и валидационную части

In [14]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from collections import Counter

In [15]:
AUG_INFO_CSV = './data/augmented_fronts_info.csv'
DATA_INFO_CSV = './data/confirmed_fronts_info.csv'
TT_INFO_CSV = './data/tt_union_fronts_info.csv'
CLASS_SIZE = 6000
RANDOM_STATE = 42

In [16]:
aug_info = pd.read_csv(AUG_INFO_CSV, sep=';')
data_info = pd.read_csv(DATA_INFO_CSV, sep=';')

In [17]:
minor_classes = ['Multicolour','Burgundy','Navy',
                 'Turquoise','Magenta','Pink',
                 'Maroon','Indigo']

# Дропаем минорные классы из датасета
for label_name in minor_classes:
    data_info = data_info.drop(index=data_info[data_info['label'] == label_name].index.tolist()).reset_index(drop=True)

In [18]:
Counter(data_info['label'])

Counter({'Black': 14317,
         'Grey': 9474,
         'White': 9395,
         'Blue': 8483,
         'Silver': 7770,
         'Red': 6095,
         'Brown': 911,
         'Green': 777,
         'Yellow': 667,
         'Beige': 600,
         'Orange': 559,
         'Purple': 362,
         'Bronze': 329,
         'Gold': 217})

In [19]:
union_info = pd.concat([aug_info, data_info]).reset_index(drop=True)

In [20]:
unique_labels = union_info['label'].unique().tolist()
balanced_data_idxs = []
for label_name in unique_labels:
    balanced_data_idxs += union_info[union_info['label'] == label_name].index.tolist()[:CLASS_SIZE]

y = union_info.iloc[balanced_data_idxs,:]['label']

print(len(balanced_data_idxs), len(y))

84000 84000


In [21]:
Counter(y)

Counter({'Gold': 6000,
         'Brown': 6000,
         'Beige': 6000,
         'Green': 6000,
         'Bronze': 6000,
         'Orange': 6000,
         'Yellow': 6000,
         'Purple': 6000,
         'Black': 6000,
         'Red': 6000,
         'White': 6000,
         'Silver': 6000,
         'Blue': 6000,
         'Grey': 6000})

In [22]:
train_idxs, test_idxs, _, _ = train_test_split(balanced_data_idxs, y, 
                                                    test_size=0.20, 
                                                    random_state=RANDOM_STATE,
                                                    shuffle=True,
                                                    stratify=y)

In [23]:
tt_info = union_info.iloc[train_idxs + test_idxs]
tt_info.loc[tt_info.index.isin(train_idxs), ['part']] = 'train'
tt_info.loc[tt_info.index.isin(test_idxs), ['part']] = 'eval'

In [24]:
tt_info['label'].unique().shape

(14,)

In [25]:
tt_info.to_csv(TT_INFO_CSV, sep=';', index=False)